In [17]:
import pandas as pd

In [18]:
df = pd.read_csv('/home/wzarzecki/ds_10000x/results/interventions/discriminant_coefs_2_100x/lambda_x_seed_changes/lambda_x_seed_thr_0p15_beta.csv', index_col=0)
# df = pd.read_csv('ratio.csv', index_col=0)

# Exclude outlier designs
outliers = {"design_69", "design_93", "design_54", "design_10"}
df = df[~df.index.isin(outliers)]

df.head()

,-5.0,-4.0,-3.0,-2.0,-1.0,0.0,1.0,2.0,3.0,4.0,5.0
design_0,0.753846,0.859649,1.188679,1.509804,3.448276,2.052632,2.250000,2.076923,2.102564,2.076923,2.305556
design_1,0.000000,0.151163,0.207317,10.000000,1.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000
design_11,0.761905,1.063830,1.016129,0.803030,1.102041,0.887097,0.934426,1.277778,1.346154,1.313725,1.214286
design_12,1.510638,0.966102,0.089109,0.796875,1.377778,3.640000,3.103448,3.103448,3.103448,3.103448,3.103448
design_13,0.000000,0.057692,0.131313,0.057143,0.036036,0.244681,0.244681,NaN,0.267442,0.445783,NaN


In [19]:
df.loc["design_0"]

-5.0    0.753846
-4.0    0.859649
-3.0    1.188679
-2.0    1.509804
-1.0    3.448276
0.0     2.052632
1.0     2.250000
2.0     2.076923
3.0     2.102564
4.0     2.076923
5.0     2.305556
Name: design_0, dtype: float64

In [20]:
import matplotlib.pyplot as plt
import os

# Ensure output directory exists
os.makedirs('plots', exist_ok=True)

# Ensure columns are numeric and sorted for x-axis
x = pd.to_numeric(df.columns)
order = x.argsort()
x = x[order]

plt.figure(figsize=(10, 6))
for row_name, row_values in df.iterrows():
    y = row_values.values[order]
    plt.plot(x, y, marker='o', linewidth=1.5, label=row_name)

plt.xlabel('lambda')
plt.ylabel('alpha helix count / beta sheet count')
plt.title('Series per design across lambda')
plt.grid(True, alpha=0.3)
plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize='small')
plt.tight_layout()
plt.savefig('plots/series_per_design_across_lambda.png', dpi=200, bbox_inches='tight')
plt.close()


/tmp/ipykernel_2257850/3907548240.py:22: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout()


In [ ]:
import matplotlib.pyplot as plt
import os

# Ensure output directory exists
os.makedirs('plots', exist_ok=True)

# Sort rows by their maximum value (descending)
sorted_rows = df.max(axis=1).sort_values(ascending=False).index

# Prepare x-axis from numeric-sorted column names
x = pd.to_numeric(df.columns)
order = x.argsort()
x = x[order]

# Create a new pane (figure) for each 10 series rows
for start in range(0, len(sorted_rows), 10):
    group = sorted_rows[start:start+10]
    plt.figure(figsize=(10, 6))
    for row_name in group:
        y = df.loc[row_name].values[order]
        plt.plot(x, y, marker='o', linewidth=1.5, label=row_name)
    plt.xlabel('lambda')
    plt.ylabel('alpha helix count / beta sheet count')
    plt.title(f'Series {start+1}-{min(start+10, len(sorted_rows))} sorted by max value')
    plt.grid(True, alpha=0.3)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize='small')
    plt.tight_layout()
    out_path = f"plots/series_sorted_by_max_value_{start+1}_{min(start+10, len(sorted_rows))}.png"
    plt.savefig(out_path, dpi=200, bbox_inches='tight')
    plt.close()


In [25]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Ensure output directory exists
os.makedirs('plots', exist_ok=True)

# Prepare x-axis from numeric-sorted column names
x = pd.to_numeric(df.columns)
order = x.argsort()
x_sorted = x[order]

# Function: longest initial strictly increasing contiguous segment length
def longest_initial_increasing_length(values: np.ndarray) -> int:
    length = 1
    for i in range(1, len(values)):
        if values[i] > values[i-1]:
            length += 1
        else:
            break
    return length

# Compute prefix lengths for each row in df
prefix_lengths = {}
for row_name, row_values in df.iterrows():
    y = row_values.values[order]
    prefix_lengths[row_name] = longest_initial_increasing_length(y)

# Sort row names by prefix length (descending)
sorted_rows_by_prefix = sorted(prefix_lengths.keys(), key=lambda r: prefix_lengths[r], reverse=True)

# Save grouped plots (10 per figure) in sorted order
for start in range(0, len(sorted_rows_by_prefix), 10):
    group = sorted_rows_by_prefix[start:start+10]
    plt.figure(figsize=(10, 6))
    for row_name in group:
        y = df.loc[row_name].values[order]
        plt.plot(x_sorted, y, marker='o', linewidth=1.5, label=f"{row_name} (L={prefix_lengths[row_name]})")
    plt.xlabel('lambda')
    plt.ylabel('alpha helix count / beta sheet count (10 if no beta sheet)')
    plt.title(f'Series {start+1}-{min(start+10, len(sorted_rows_by_prefix))} sorted by longest initial increasing length')
    plt.grid(True, alpha=0.3)
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', fontsize='small')
    plt.tight_layout()
    out_path = f"plots/series_sorted_by_initial_increasing_length_{start+1}_{min(start+10, len(sorted_rows_by_prefix))}.png"
    plt.savefig(out_path, dpi=200, bbox_inches='tight')
    plt.close()

# Also export the computed lengths as a CSV for reference
pd.Series(prefix_lengths, name='longest_initial_increasing_length') \
  .sort_values(ascending=False) \
  .to_csv('plots/longest_initial_increasing_length.csv')



In [23]:
df.loc["design_49"]

-5.0    0.060000
-4.0    0.062500
-3.0    0.231579
-2.0    0.648649
-1.0    1.822222
0.0     0.950820
1.0     1.904762
2.0     1.975000
3.0     1.950000
4.0          NaN
5.0     2.025000
Name: design_49, dtype: float64